In [2]:
:dep ndarray = "^0.15.0"
:dep plotters = { version = "^0.3.4", default_features = false, features = ["evcxr", "all_series"] }

use ndarray::{array, Array, Array1, Array2, ArrayBase, Ix1, Ix2, Ix3, Zip};
extern crate plotters;
use plotters::prelude::*;

## Perceptron Logic gate

$$y = \begin{cases}0 \quad (w_1x_1+w_2x_2\le\theta)\\1 \quad (w_1x_2+w_2x_2>\theta)\end{cases}$$

In [3]:
fn simple_AND(x1: f32, x2: f32) -> f32 {
    let w1 = 0.5;
    let w2 = 0.5;
    let theta = 0.7_f32;
    let temp = x1 * w1 + x2 * w2;
    if temp <= theta {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [4]:
assert!(simple_AND(0., 0.) == 0.);
assert!(simple_AND(1., 0.) == 0.);
assert!(simple_AND(0., 1.) == 0.);
assert!(simple_AND(1., 1.) == 1.);

$$y = \begin{cases}0 \quad (b+w_1x_1+w_2x_2\le0)\\1 \quad (b+w_1x_2+w_2x_2>0)\end{cases}$$

$$y = h(b+w_1x_1+w_2x_2)$$

$$h(x) = \begin{cases}0 \quad (x\le0)\\1 \quad (x>0)\end{cases}$$

In [5]:
fn AND(x1: f32, x2: f32) -> f32 {
    let x = array![x1, x2];
    let weight = array![0.5_f32, 0.5];
    let bias = -0.7_f32;
    let temp = x.dot(&weight) + bias;
    if temp <= 0. {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [6]:
assert!(AND(0., 0.) == 0.);
assert!(AND(1., 0.) == 0.);
assert!(AND(0., 1.) == 0.);
assert!(AND(1., 1.) == 1.);

In [7]:
fn NAND(x1: f32, x2: f32) -> f32 {
    let x = array![x1, x2];
    let weight = array![-0.5_f32, -0.5];
    let bias = 0.7_f32;
    let temp = x.dot(&weight) + bias;
    if temp <= 0. {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [8]:
assert!(NAND(0., 0.) == 1.);
assert!(NAND(1., 0.) == 1.);
assert!(NAND(0., 1.) == 1.);
assert!(NAND(1., 1.) == 0.);

In [9]:
fn OR(x1: f32, x2: f32) -> f32 {
    let x = array![x1, x2];
    let weight = array![0.5_f32, 0.5];
    let bias = -0.2_f32;
    let temp = x.dot(&weight) + bias;
    if temp <= 0. {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [10]:
assert!(OR(0., 0.) == 0.);
assert!(OR(1., 0.) == 1.);
assert!(OR(0., 1.) == 1.);
assert!(OR(1., 1.) == 1.);

In [11]:
fn XOR(x1: f32, x2: f32) -> f32 {
    let s1 = NAND(x1, x2);
    let s2 = OR(x1, x2);
    AND(s1, s2)
}

In [12]:
assert!(XOR(0., 0.) == 0.);
assert!(XOR(1., 0.) == 1.);
assert!(XOR(0., 1.) == 1.);
assert!(XOR(1., 1.) == 0.);

## Activation Function

$$a=b+x_1x_1+w_2x_2$$

$$y=h(a)$$

In [13]:
fn draw_figure(mut x_data: Array1<f32>, y_data: Array1<f32>, xlim: (f32, f32), ylim: (f32, f32)) {
    evcxr_figure((480, 320), |root| {
        root.fill(&WHITE)?;
        let mut chart = ChartBuilder::on(&root)
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(xlim.0..xlim.1, ylim.0..ylim.1)?;

        chart.configure_mesh().disable_mesh().draw()?;

        chart.draw_series(LineSeries::new(
            Zip::from(&mut x_data).and(&y_data).map_collect(|x, y| (*x, *y)),
            &BLACK
        ));
        Ok(())
    }).evcxr_display()
}

In [14]:
fn simple_step(x: f32) -> i32 {
    if x > 0. {1} else {0}
}

assert!(simple_step(0.1) == 1);
assert!(simple_step(-0.1) == 0);
assert!(simple_step(0.5) == 1);
assert!(simple_step(2.3) == 1);

fn step_array(arr_x: &Array1<f32>) -> Array1<f32> {
    arr_x.map(|x| if *x > 0. {1.} else {0.})
}

assert!(step_array(&array![-1.0_f32, 1.0, 2.0]) == array![0.0_f32, 1.0, 1.0]);
let x = Array::range(-5.0_f32, 5.0, 0.1);
let y = step_array(&x);
draw_figure(x, y, (-6.0, 6.0), (-0.1, 1.1))

()

In [15]:
fn sigmoid_array(arr_x: &Array1<f32>) -> Array1<f32> {
    arr_x.map(|x| 1. / (1. + (-x).exp()))
}

assert!(sigmoid_array(&array![-1.0, 1.0, 2.0]) == array![0.26894143, 0.7310586, 0.880797]);

let x = Array::range(-5.0_f32, 5.0, 0.1);
let y = sigmoid_array(&x);
draw_figure(x, y, (-6.0, 6.0), (-0.1, 1.1))

()

In [16]:
fn relu(arr_x: &Array1<f32>) -> Array1<f32> {
    arr_x.map(|x| x.max(0.))
}

assert!(relu(&array![-1.0, 0.5, 2.0]) == array![0., 0.5, 2.0]);

let x = Array::range(-5.0_f32, 5.0, 0.1);
let y = relu(&x);
draw_figure(x, y, (-6.0, 6.0), (-1.1, 5.1))

()